In [ ]:
%%capture

# Comment this out if you don't want to install pennylane from this notebook
!pip install pennylane

# Comment this out if you don't want to install matplotlib from this notebook
!pip install matplotlib

# Differentiable quantum computing with PennyLane

In this tutorial we will:

* learn step-by-step how quantum computations are implemented in PennyLane,
* understand parameter-dependent quantum computations ("variational circuits"), 
* build our first quantum machine learning model, and
* compute its gradient.

We need the following imports:

In [2]:
import pennylane as qml
from pennylane import numpy as np

## 1. Quantum nodes

In PennyLane, a *quantum node* is a computational unit that involves the construction, evaluation, pre- and postprocessing of quantum computations.

A quantum node consists of a *quantum function* that defines a circuit, as well as a *device* on which it is run. 

There is a growing [device ecosystem](https://pennylane.ai/plugins.html) which allows you to change only one line of code to dispatch your quantum computation to local simulators, remote simulators and remote hardware from different vendors.

Here we will use the built-in `default.qubit` device.

In [3]:
dev = qml.device('default.qubit', wires=2)

To combine the device with a quantum function to a quantum node we can use the `qml.qnode` decorator. The function can then be evaluated as if it was any other python function. Internally, it will construct a circuit and run it on the device.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.Hadamard(wires=0)
    return qml.probs(wires=[0, 1])

circuit()

## 2. Building quantum circuits

### The initial state

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/raw/7738ba7cea0c6019dadf6f7f7cd782291f1add2a/figures/1.png" width="500" height="100">
<br />

The initial state has 100% probability to be measured in the "0..0" configuration. Let's see how we can verify this with PennyLane.

In [ ]:
@qml.qnode(dev)
def circuit():
    return qml.probs(wires=[0, 1])

circuit()

The internal state vector that we use to mathematically keep track of probabilities is complex-valued. Since `default.qubit` is a simulator we can have a look at the state, for example by checking the device's `state` attribute.

In [ ]:
dev.state

### Unitary evolutions

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/raw/7738ba7cea0c6019dadf6f7f7cd782291f1add2a/figures/2.png" width="500">
<br />

Quantum circuits are represented by unitary matrices. We can evolve the initial state by an arbitrary unitrary matrix as follows:

In [ ]:
s = 1/np.sqrt(2)
U = np.array([[0., -s, 0.,  s],
              [ s, 0., -s, 0.],
              [ s, 0.,  s, 0.],
              [0., -s, 0., -s]])

@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.probs(wires=[0, 1])

circuit()

The internal quantum state changed.

In [ ]:
dev.state

### Measurements sample outcomes from the distribution

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/raw/7738ba7cea0c6019dadf6f7f7cd782291f1add2a/figures/3.png" width="500">
<br />

The most common measurement takes samples $-1, 1$ from the "Pauli-Z" observable. The samples indicate if the qubit was measured in state $| 0 \rangle$ or $| 1 \rangle$.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.sample(qml.PauliZ(wires=0)), qml.sample(qml.PauliZ(wires=1))

circuit()

The quantum state should be still the same as above.

In [ ]:
dev.state

### Computing expectation values 

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/raw/7738ba7cea0c6019dadf6f7f7cd782291f1add2a/figures/4.png" width="500">
<br />

When we want outputs of computations to be deterministic, we often interpret the expected measurement outcome as the result. This value is estimated by taking lots of samples and averaging over them.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.QubitUnitary(U, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

Again, the quantum state should be the same as above.

In [ ]:
dev.state

### Quantum circuits are decomposed into gates

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/raw/7738ba7cea0c6019dadf6f7f7cd782291f1add2a/figures/5.png" width="500">
<br />

Quantum circuits rarely consist of one large unitary (which quickly becomes intractably large as the number of qubits grow). Instead, they are composed of *quantum gates*.

In [ ]:
@qml.qnode(dev)
def circuit():
    qml.PauliX(wires=0)
    qml.CNOT(wires=[0,1])
    qml.Hadamard(wires=0)
    qml.PauliZ(wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit()

### Some gates depend on "control" parameters

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/raw/7738ba7cea0c6019dadf6f7f7cd782291f1add2a/figures/6.png" width="500">
<br />

To train circuits, there is a special subset of gates which is of particular interest: the Pauli rotation gates. These "rotate" a special representation of the quantum state around a specific axis. The gates depend on a scalar parameter which is the angle of the rotation. 

In [4]:
@qml.qnode(dev)
def circuit(w1, w2):
    qml.RX(w1, wires=0)
    qml.CNOT(wires=[0,1])
    qml.RY(w2, wires=1)
    return qml.expval(qml.PauliZ(wires=0)), qml.expval(qml.PauliZ(wires=1))

circuit(0.2, 1.3)

## this is a variational circuit that changes output based on inputs

array([0.98006658, 0.26216666])

The names `w1`, `w2` are already suggestive that these can be used like the trainable parameters of a classical machine learning model. But we could also call the control parameters `x1`, `x2` and encode data features into quantum states. 

## 3. A full quantum machine learning model and its gradient

Finally, we can use pre-coded routines or [templates](https://pennylane.readthedocs.io/en/stable/introduction/templates.html) to conveniently build full quantum machine learning model that include a data encoding part, and a trainable part.

<br />
<img src="https://github.com/XanaduAI/pennylane-demo-cern/raw/7738ba7cea0c6019dadf6f7f7cd782291f1add2a/figures/7.png" width="500">
<br />

Here, we will use the `AngleEmbedding` template to load the data, and the `BasicEntanglingLayers` as the trainable part of the circuit.

In [5]:
@qml.qnode(dev)
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1]) #green block
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1]) # purple block
    return qml.expval(qml.PauliZ(wires=0))


x = np.array([0.1, 0.2], requires_grad=True)
w = np.array([[-2.1, 1.2], [-1.4, -3.9], [0.5, 0.2]])

quantum_model(x, w)

0.012023000064217415

We can draw the circuit.

In [6]:
print(quantum_model.draw())

 0: ──RX(0.1)──RX(-2.1)──╭C──RX(-1.4)──╭C──RX(0.5)──╭C──┤ ⟨Z⟩ 
 1: ──RX(0.2)──RX(1.2)───╰X──RX(-3.9)──╰X──RX(0.2)──╰X──┤     



The best thing is that by using PennyLane, we can easily compute its gradient!

In [ ]:
gradient_fn = qml.grad(quantum_model)

gradient_fn(x, w)

This allows us to slot the quantum circuit into the machine learning example from the previous notebook.

#  TASKS 

1. Copy and paste the code from the previous notebook to here and replace the classical model by 
   the `quantum_model` function. This will allow you to train the model!

2. Add a bias term to the quantum model.

3. Replace the hand-coded optimisation step by a native [PennyLane optimiser](https://pennylane.readthedocs.io/en/stable/introduction/optimizers.html).

4. Rewrite the entire example in PyTorch. 

   Tipp: You must set the qnode to the correct interface via `@qml.qnode(dev, interface='tf')`.

In [7]:
n_samples = 100
X0 = np.array([[np.random.normal(loc=-1, scale=1), 
                np.random.normal(loc=1, scale=1)] for i in range(n_samples//2)]) 
X1 = np.array([[np.random.normal(loc=1, scale=1), 
                np.random.normal(loc=-1, scale=1)] for i in range(n_samples//2)]) 
X = np.concatenate([X0, X1], axis=0)
Y = np.concatenate([-np.ones(50), np.ones(50)], axis=0)
data = list(zip(X, Y))

def loss(a, b):
    return (a - b)**2

def average_loss(w, data):
    c = 0 
    for x,y in data:
        prediction = quantum_model(x, w)
        c += loss(prediction, y)
    return c/len(data)


w = np.array([[-2.1, 1.2], [-1.4, -3.9], [0.5, 0.2]])
average_loss(w, data)


tensor(0.8389, requires_grad=True)

In [8]:

#quantum_model(x, w)

gradient_fn = qml.grad(average_loss, argnum=0)
gradient_fn(w, data)

w_new = w - 0.05*gradient_fn(w, data)
average_loss(w_new, data)



tensor(0.80323806, requires_grad=True)

In [11]:
@qml.qnode(dev)
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1]) #green block
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1]) # purple block
    return qml.expval(qml.PauliZ(wires=0))

def model (x, w, b):
        q_pred = quantum_model(x,w)
        return q_pred + b

def loss(a, b):
    return (a - b)**2

def average_loss(w, b, data):
    c = 0 
    for x,y in data:
        prediction = model(x, w, b) #scalar constant $$ f(x, w) = \langle w, x \rangle + b, $$
        c += loss(prediction, y)
    return c/len(data)

gradient_fn = qml.grad(average_loss, argnum=[0,1])

#x = np.array([0.1, 0.2], requires_grad=True)
w = np.array([[-2.1, 1.2], [-1.4, -3.9], [0.5, 0.2]])
b = np.array(np.random.random(size=(1,)))

historyb = []
historyw = []
for i in range(10):
    w_new = w - 0.05*gradient_fn(w, b ,data)[0]
    b_new = b - 0.05*gradient_fn(w, b, data)[1]
    print(average_loss(w_new, b_new , data))
    historyw.append(w_new)
    historyb.append(b_new)
    w = w_new
    b = b_new

[1.07968181]
[0.99011382]
[0.91508985]
[0.85223334]
[0.79955027]
[0.75535903]
[0.7182392]
[0.68699252]
[0.66061151]
[0.638253]


In [ ]:
print("Lowest Cost Parameters\n=================")
print("W:\n{}".format(historyw[len(historyw)-1]))
print("B:\n{}".format(historyb[len(historyb)-1]))
print("Cost: {}".format(average_loss(w_new, b_new , data)))

In [ ]:
def model (x, w, b):
        q_pred = quantum_model(x,w)
        return q_pred + b

In [ ]:
model(x,w,0.1)

In [ ]:
gradient_fn = qml.grad(model, argnum=[1,2])

In [ ]:
gradient_fn(x,w,0.1)

3. Replace the hand-coded optimisation step by a native [PennyLane optimiser](https://pennylane.readthedocs.io/en/stable/introduction/optimizers.html).

In [33]:
n_samples = 100
X0 = np.array([[np.random.normal(loc=-1, scale=1), 
                np.random.normal(loc=1, scale=1)] for i in range(n_samples//2)]) 
X1 = np.array([[np.random.normal(loc=1, scale=1), 
                np.random.normal(loc=-1, scale=1)] for i in range(n_samples//2)]) 
X = np.concatenate([X0, X1], axis=0)
Y = np.concatenate([-np.ones(50), np.ones(50)], axis=0)
data = list(zip(X, Y))

@qml.qnode(dev)
def quantum_model(x, w):
    qml.templates.AngleEmbedding(x, wires=[0, 1]) 
    qml.templates.BasicEntanglerLayers(w, wires=[0, 1])
    return qml.expval(qml.PauliZ(wires=0))

def model (x, w, b):
        q_pred = quantum_model(x,w)
        return q_pred + b

def average_loss(w, data, b):
    c = 0 
    for x,y in data:
        prediction = model(x, w, b)
        c += (prediction - y)**2
    return c/len(data)

w = np.array([[-2.1, 1.2], [-1.4, -3.9], [0.5, 0.2]])
b = np.array(np.random.random(size=(1,)))
params = [w,data,b]

# initialise the optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.4) 

history_p = []
history_c = []

for i in range(15):
    params, prev_cost = opt.step_and_cost(lambda params: average_loss(params[0],params[1],params[2]),params)
    history_p.append(params)
    history_c.append(prev_cost)

    cost = average_loss(params[0],params[1],params[2])
    print("===============")
    print(params[0]) # w
    print(params[2]) # b
    print(cost)
    print("---------------")
    

[[-2.11349981  1.2       ]
 [-1.05569924 -3.9       ]
 [ 0.48650019  0.2       ]]
[0.07060832]
[0.58376828]
---------------
[[-2.11769361  1.2       ]
 [-0.86478759 -3.9       ]
 [ 0.48230639  0.2       ]]
[-0.0059329]
[0.49161655]
---------------
[[-2.11366025  1.2       ]
 [-0.74914559 -3.9       ]
 [ 0.48633975  0.2       ]]
[-0.02656118]
[0.45313387]
---------------
[[-2.11014523  1.2       ]
 [-0.67076722 -3.9       ]
 [ 0.48985477  0.2       ]]
[-0.03536021]
[0.43007504]
---------------
[[-2.10837471  1.2       ]
 [-0.61322721 -3.9       ]
 [ 0.49162529  0.2       ]]
[-0.0404583]
[0.41321572]
---------------
[[-2.10778257  1.2       ]
 [-0.56856244 -3.9       ]
 [ 0.49221743  0.2       ]]
[-0.04358371]
[0.39936139]
---------------
[[-2.10786304  1.2       ]
 [-0.53246677 -3.9       ]
 [ 0.49213696  0.2       ]]
[-0.04549676]
[0.38717981]
---------------
[[-2.10831048  1.2       ]
 [-0.50239699 -3.9       ]
 [ 0.49168952  0.2       ]]
[-0.04665422]
[0.37604055]
---------------
[[-

4. Rewrite the entire example in PyTorch. 

   Tipp: You must set the qnode to the correct interface via `@qml.qnode(dev, interface='tf')`.

In [ ]:
import torch

data = [[torch.tensor(x), torch.tensor(y)] for x, y in data]

def loss(a, b):
    return (a - b)**2

def sigmoid(z):
    return 1/(1 + torch.exp(-z))

def model_torch(x, w):
    return torch.dot(x,w) 

def average_loss(w, data):
    c = 0
    for x, y in data:
        prediction = model_torch(x, w)
        c += loss(prediction, y)
    return c/len(data)

w_init = np.random.random(size=(2,))

w = torch.tensor(w_init, requires_grad=True)

opt = torch.optim.Adam([w], lr = 0.1)

# One way of optimising is to use closures
def closure():
    opt.zero_grad()
    loss = average_loss(w, data)
    loss.backward()
    return loss

for i in range(15):
    opt.step(closure)